In [ ]:
### ML ASSIGNMENT

In [ ]:
### Get location

In [682]:
# import libraries

import boto3, re, sys, math, json, os, sagemaker, urllib.request
from sagemaker import get_execution_role
import numpy as np                                
import pandas as pd                               
import matplotlib.pyplot as plt                   
from IPython.display import Image                 
from IPython.display import display               
from time import gmtime, strftime                 
from sagemaker.predictor import csv_serializer   

# Define IAM role

role = get_execution_role()
prefix = 'house_price_xgboost-dm'
containers = {'us-west-2': '433757028032.dkr.ecr.us-west-2.amazonaws.com/xgboost:latest',
              'us-east-1': '811284229777.dkr.ecr.us-east-1.amazonaws.com/xgboost:latest',
              'us-east-2': '825641698319.dkr.ecr.us-east-2.amazonaws.com/xgboost:latest',
              'eu-west-1': '685385470294.dkr.ecr.eu-west-1.amazonaws.com/xgboost:latest'} # each region has its XGBoost container
my_region = boto3.session.Session().region_name # set the region of the instance
print("Success - the MySageMakerInstance is in the " + my_region + " region. You will use the " + containers[my_region] + " container for your SageMaker endpoint.")

NoCredentialsError: Unable to locate credentials

In [ ]:
### create bucket - if not allready created?

In [ ]:
%%time
bucket_name = 'mutest-01' # <--- CHANGE THIS VARIABLE TO A UNIQUE NAME FOR YOUR BUCKET
s3 = boto3.resource('s3')
try:
    if  my_region == 'eu-west-1':
      s3.create_bucket(Bucket=bucket_name)
    else: 
      s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={ 'LocationConstraint': my_region })
    print('S3 bucket created successfully')
except Exception as e:
    print('S3 error: ',e)

In [ ]:
#### CHECK contents of bucket #####
from boto3 import client
conn = client('s3')  # again assumes boto.cfg setup, assume AWS S3
for key in conn.list_objects(Bucket='mutest-01')['Contents']:
    print(key['Key'])

In [ ]:
### download data and load data into dataframe

In [ ]:
%%time

url = 'http://prod1.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com/pp-monthly-update-new-version.csv'

try:
  urllib.request.urlretrieve (url)
  print('pp-monthly-update-new-version.csv')
except Exception as e:
  print('Data load error: ',e)

try:
  model_data = pd.read_csv('./house_price_data//pp-monthly-update-new-version.csv',delimiter=',')
  print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)

In [ ]:
### have a look at the data
model_data.head(2)

In [ ]:
model_data.info()

In [ ]:
### remove unwanted cols
cols = [0,3,5,7,8,9,10,11,12,14,15] ### remove unwanted cols
model_data.drop(model_data.columns[cols],axis=1,inplace=True)

In [ ]:
### rename cols
model_data.rename(columns={ model_data.columns[0]: "price", model_data.columns[1]: "date", model_data.columns[2]: "type",model_data.columns[3]: "lease", model_data.columns[4]: "location" }, inplace = True)

In [ ]:
### have a look
model_data.head(2)

In [ ]:
### encode and lable non numeric cols 
### house type and lease type
### one hot encoding

from sklearn.preprocessing import LabelEncoder
h_type = LabelEncoder()
h_lease = LabelEncoder()
model_data['type_encoded'] = h_type.fit_transform(model_data.type)
model_data['lease_encoded'] = h_lease.fit_transform(model_data.lease)

from sklearn.preprocessing import OneHotEncoder
type_ohe = OneHotEncoder()
lease_ohe = OneHotEncoder()
X = type_ohe.fit_transform(model_data.type_encoded.values.reshape(-1,1)).toarray()
Xm = lease_ohe.fit_transform(model_data.lease_encoded.values.reshape(-1,1)).toarray()

dfOneHot = pd.DataFrame(X, columns = ["type_"+str(int(i)) for i in range(X.shape[1])])
model_data = pd.concat([model_data, dfOneHot], axis=1)

dfOneHot = pd.DataFrame(Xm, columns = ["lease_"+str(int(i)) for i in range(Xm.shape[1])])
##model_data = pd.concat([model_data, dfOneHot], axis=1)


In [ ]:
### have a look
model_data.head(10)

In [ ]:
### rename one hot encoded cols and convert all to ints
model_data.rename(columns={ model_data.columns[4]: "location_london", model_data.columns[7]: "type_D", model_data.columns[8]: "type_F", model_data.columns[9]: "type_O",model_data.columns[10]: "type_S", model_data.columns[11]: "type_T" }, inplace = True)
### date to year to int
model_data["date"] = pd.DatetimeIndex(model_data["date"]).year.astype(int)
### convert to int
model_data = model_data.astype(int)

In [ ]:
### have a look
model_data.head(10)

In [ ]:
### location in or out of London - LONDON = 1 NOT LONDON = 0
model_data['location_london'] = model_data['location_london'].apply(lambda x: 1 if x == 'GREATER LONDON' else 0)

In [ ]:
### have a look
model_data.head(10)

In [ ]:
### delete not used cols
model_data = model_data.drop(model_data.columns[[2,3,5]], axis=1) 

In [ ]:
### have a look
model_data.head(10)

In [ ]:
### test . train split <2017 train data. 2017 test data

In [ ]:
model_data = model_data.sort_values(by=["date"], ascending=False)

In [ ]:
model_data.tail(10)

In [ ]:
model_data.info()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.pairplot(model_data, hue='date');

In [ ]:
model_data.describe()

In [ ]:
df_rat_by_year = model_data['price']
plt.scatter(df_rat_by_year.index, df_rat_by_year)
plt.xlabel('Yr')
plt.ylabel('£ 00,000,000');

In [ ]:
### train test validate
### jey date 2017

model_data_test = model_data[model_data['date'] == 2017] 
model_data_train = model_data[model_data['date'] < 2017] 
model_data_validate = model_data[model_data['date'] > 2017] 


In [ ]:
##model_data_validate

In [ ]:
### setting up completed
### now training

In [ ]:
### data download and cleaning completed
### have a look
!ls house_price_data/
### next prepare the data for training

In [ ]:
%%time
### this was done above - the splitting
### train_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data))])
### print(train_data.shape, test_data.shape)

In [ ]:
%%time
pd.concat([train_data['y_yes'], train_data.drop(['y_no', 'y_yes'], axis=1)], axis=1).to_csv('train.csv', index=False, header=False)

In [ ]:
%%time
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')

In [ ]:
%%time
s3_input_train = sagemaker.s3_input(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')

In [ ]:
%%time
sess = sagemaker.Session()
xgb = sagemaker.estimator.Estimator(containers[my_region],role, train_instance_count=1, train_instance_type='ml.m4.xlarge',output_path='s3://{}/{}/output'.format(bucket_name, prefix),sagemaker_session=sess)
xgb.set_hyperparameters(max_depth=5,eta=0.2,gamma=4,min_child_weight=6,subsample=0.8,silent=0,objective='binary:logistic',num_round=100)

In [ ]:
%%time
s3_input_train

In [ ]:
%%time
xgb.fit({'train': s3_input_train})

In [ ]:
%%time
xgb_predictor = xgb.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge')

In [ ]:
%%time
test_data_array = test_data.drop(['y_no', 'y_yes'], axis=1).values #load the data into an array
xgb_predictor.content_type = 'text/csv' # set the data type for an inference
xgb_predictor.serializer = csv_serializer # set the serializer type
predictions = xgb_predictor.predict(test_data_array).decode('utf-8') # predict!
predictions_array = np.fromstring(predictions[1:], sep=',') # and turn the prediction into an array
print(predictions_array.shape)

In [ ]:
%%time
cm = pd.crosstab(index=test_data['y_yes'], columns=np.round(predictions_array), rownames=['Observed'], colnames=['Predicted'])
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No Purchase", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Purchase", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))

In [ ]:
### get rid of everything
### clean up
### remove bucket contents

In [ ]:
%%time
sagemaker.Session().delete_endpoint(xgb_predictor.endpoint)


In [ ]:
%%time
bucket_to_delete = boto3.resource('s3').Bucket(bucket_name)

In [ ]:
%%time
bucket_to_delete.objects.all().delete()

In [ ]:
### end